In [1]:
import read_muram as rmu
import dp_plot_tools as dplt
import muram_eos as eos
import numpy as np

Importing read_muram.py
Importing dp_plot_tools.py
Importing muram_eos.py


In [2]:
## Load the EOS file
eos_file = "../RUNDIR/Uppsala_mergedeos_PI_A.dat"
mu_eos=eos.mu_eos(eos_file)

In [3]:
## Set the simulation size and range
## Iteration number

iter = 0

## Nz = vertical direction (dimension 0 in MURaM)
## Nx = first horizontal direction (set = 1 for 1D)
## Ny = second horizontal direection (set = 1 for 2D or 1D)

Nz = 400
Nx = 2400
Ny = 1

## z_box_top = the top height above the surface in Mm
## z_box_range = the vertical extend in Mm

z_box_top = 1.0
z_box_range = 4.0

x_box_range = 24.0
y_box_range = 24.0

dz = z_box_range/Nz
dx = x_box_range/Nx
dy = y_box_range/Ny
## Output Directory

out_dir = "/ptmp/damp/2D_SCATTER_TESTS/initial_convection/"

In [4]:
## Load the SSM Background
N_SSM = 2480
z_SSM = -1.0e-6*np.loadtxt('tables/R_SSM.dat',dtype=np.float).ravel()[::-1]
r_SSM = np.loadtxt('tables/rho_SSM.dat',dtype=np.float).ravel()[::-1]
t_SSM = np.loadtxt('tables/T_SSM.dat',dtype=np.float).ravel()[::-1]
p_SSM = np.loadtxt('tables/P_SSM.dat',dtype=np.float).ravel()[::-1]

In [5]:
## bkg values
z_bkg = z_box_top-((Nz-1)-np.arange(Nz,dtype=np.float))*np.float(z_box_range)/np.float(Nz)

## Interpolate for rho, pressure and density
r_bkg = np.exp(np.interp(z_bkg,z_SSM,np.log(r_SSM)))
p_bkg = np.exp(np.interp(z_bkg,z_SSM,np.log(p_SSM)))
t_bkg = np.exp(np.interp(z_bkg,z_SSM,np.log(t_SSM)))

## Above the SSM use an isothermal, constant pressure scale height extrapolation

r0 = r_SSM[-1]
p0 = p_SSM[-1]
t0 = t_SSM[-1]
z0 = z_SSM[-1]
H0 = p0/r0/2.74e4

for kk in range(Nz):
    if(z_bkg[kk]> z_SSM[-1]):
      r_bkg[kk] = r0*np.exp((z0-z_bkg[kk])*1.0e8/H0)
      p_bkg[kk] = p0*np.exp((z0-z_bkg[kk])*1.0e8/H0)
      t_bkg[kk] = t0

## Invert temperature table for internal energy
e_bkg=np.zeros(Nz)
for ii in range(Nz):
    T_array = mu_eos.interp_T(np.exp(mu_eos.xeps)-mu_eos.eps_off,np.tile(r_bkg[ii],mu_eos.n_eps))
    Tisnan = np.isnan(T_array)
    e_bkg[ii]= np.exp(np.interp(t_bkg[ii],T_array[~Tisnan],mu_eos.xeps[~Tisnan]))-mu_eos.eps_off
    
    
s_bkg = mu_eos.interp_s(e_bkg,r_bkg)

In [6]:
## Bottom boundary conditions
zstag= z_bkg[0]-0.5*z_box_range/np.float(Nz)

rbc  = np.exp(np.interp(zstag,z_SSM,np.log(r_SSM)))
pbc  = np.exp(np.interp(zstag,z_SSM,np.log(p_SSM)))
Tbc  = np.exp(np.interp(zstag,z_SSM,np.log(t_SSM)))

In [8]:
## Now write output result_prim files

#Density
(np.tile(r_bkg,(Ny,Nx,1)).ravel().astype(np.single)).tofile(out_dir+'3D/result_prim_0.'+str(iter).zfill(6))
#Velocities
(2.0e2*(np.random.rand(Nx*Ny*Nz)-0.5).astype(np.single)).tofile(out_dir+'3D/result_prim_1.'+str(iter).zfill(6))
(2.0e2*(np.random.rand(Nx*Ny*Nz)-0.5).astype(np.single)).tofile(out_dir+'3D/result_prim_2.'+str(iter).zfill(6))
(2.0e2*(np.random.rand(Nx*Ny*Nz)-0.5).astype(np.single)).tofile(out_dir+'3D/result_prim_3.'+str(iter).zfill(6))
# Internal Energy
(np.tile(e_bkg*r_bkg,(Ny,Nx,1)).ravel().astype(np.single)).tofile(out_dir+'3D/result_prim_4.'+str(iter).zfill(6))
# Magnetic Fields
np.zeros(Nx*Ny*Nz,dtype=np.single).tofile(out_dir+'3D/result_prim_5.'+str(iter).zfill(6))
np.zeros(Nx*Ny*Nz,dtype=np.single).tofile(out_dir+'3D/result_prim_6.'+str(iter).zfill(6))
np.zeros(Nx*Ny*Nz,dtype=np.single).tofile(out_dir+'3D/result_prim_7.'+str(iter).zfill(6))
# Heat Flux
np.zeros(Nx*Ny*Nz,dtype=np.single).tofile(out_dir+'3D/result_prim_8.'+str(iter).zfill(6))

# Header
f=open(out_dir + '3D/Header.'+str(iter).zfill(6),'w')
f.write(str(np.int(Nz)) + "  " + str(np.int(Nx)) + "  " + str(np.int(Ny)) + "  " + str(np.single(dz)) + "  " + str(np.single(dx)) + "  " + str(np.single(dy)) + " " + str(np.single(0))  + " " + str(np.single(0)) + " " + str(np.single(2.99792458e10)))
f.close()

#Backup.dat
f=open(out_dir + 'backup.dat','w')
f.write(str(np.int(0)) + "  " + str(np.int(0)) + "  " + str(np.single(pbc)) + "  " + str(np.single(s_bkg[0])) + "  " + str(np.single(r_bkg.sum()*Nx*Ny)))
f.close()

In [9]:
r_bkg.min()

2.6843044126214637e-11